In [ ]:
import keras

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models, optimizers, initializers

In [ ]:
download = drive.CreateFile({'id': '1X2HrMOY5q9KNOndpcMPjmAdW5M83p8Nb'})    # Pulling the dataset from Google Drive (Use the file id of the dataset to be used)
download.GetContentFile('Dataset')          

In [ ]:
!7z x Dataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 242191542 bytes (231 MiB)

Extracting archive: Dataset
  7% 4096 Open              --
Path = Dataset
Type = zip
Physical Size = 242191542

  0%      1% 561 - Split Dataset/17/123/a_a_a_bcc000024.bmp                                                     2% 1183           2% 1728 - Split Dataset/17/68/a_a_a_bcc000177.bmp                                                     3% 2353 - Split Dataset/17/148/a_a_a_a_a_bcc007050.bmp

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.utils import plot_model

In [ ]:
mv "Split Dataset" "Split_Dataset"

In [ ]:
builder = tfds.ImageFolder('./Split_Dataset')

In [ ]:
base_dir = "./Split_Dataset"
train_dir = os.path.join(base_dir, "")

In [ ]:
def fn(x):
  x=(x-127.50)/127.50
  return x

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(preprocessing_function=fn)

In [ ]:
train_gen_list = []
n_of_classes = 10
class_num_for_folder = []
for i in range(17):
  train_dir_cur = os.path.join(train_dir + str(i + 1))
  train_gen_list.append(train_datagen.flow_from_directory(train_dir_cur, batch_size = 64, class_mode = 'sparse', color_mode = 'grayscale', target_size = (64, 64)))
  if (i < 16):
    class_num_for_folder.append(10)
class_num_for_folder.append(11)


Found 2808 images belonging to 10 classes.
Found 3190 images belonging to 10 classes.
Found 3339 images belonging to 10 classes.
Found 3622 images belonging to 10 classes.
Found 3041 images belonging to 10 classes.
Found 3343 images belonging to 10 classes.
Found 2908 images belonging to 10 classes.
Found 3178 images belonging to 10 classes.
Found 3297 images belonging to 10 classes.
Found 3031 images belonging to 10 classes.
Found 3045 images belonging to 10 classes.
Found 3282 images belonging to 10 classes.
Found 3205 images belonging to 10 classes.
Found 3281 images belonging to 10 classes.
Found 3334 images belonging to 10 classes.
Found 3267 images belonging to 10 classes.
Found 3055 images belonging to 11 classes.


In [ ]:
import tensorflow as tf

In [ ]:
inp_size = 64
inp_shape = (64, 64, 1)
def get_generator(num_classes):   
    input_noise = keras.layers.Input(shape = (110, ))
    input_label = keras.layers.Input(shape = (1, ))
    initial_dim_noise = 4 * 4 * 384
    initial_dim_label = 16
    initial_shape_noise = (4, 4, 384)
    initial_shape_label = (4, 4, 1)
    init = initializers.random_normal(stddev = 0.02)
    x2 = layers.Embedding(num_classes, 50)(input_label)
    x1 = layers.Dense(initial_dim_noise)(input_noise)
    x1 = layers.Activation('relu')(x1)
    x2 = layers.Dense(initial_dim_label)(x2)
    x1 = layers.Reshape(initial_shape_noise)(x1)
    x2 = layers.Reshape(initial_shape_label)(x2)
    x = layers.Concatenate(axis = -1)([x1, x2])
    x = layers.Conv2DTranspose(384, (5, 5), (2, 2), padding = 'same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(256, (5, 5), (2, 2), padding = 'same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(192, (5, 5), (2, 2), padding = 'same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2DTranspose(1, (5, 5), (2, 2), padding = 'same', kernel_initializer=init)(x)
    x = layers.Activation('tanh')(x)
    model = models.Model(inputs = [input_noise, input_label], outputs = x)
    return model
def get_discriminator(num_classes):
    input = layers.Input(shape = inp_shape)
    true_val = layers.Input(shape = (1,))
    mid = layers.Embedding(num_classes, 50)(true_val)
    mid = layers.Dense(512)(mid)
    mid = layers.Dense(64*64)(mid)
    mid = layers.Reshape(inp_shape)(mid);
    x = layers.Concatenate(axis = -1)([input, mid])
    init = initializers.RandomNormal(stddev = 0.02)
    x = layers.Conv2D(16, (3, 3), (2, 2), padding = 'same', kernel_initializer = init)(input)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(32, (3, 3), (1, 1), padding = 'same', kernel_initializer = init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(64, (3, 3), (2, 2), padding = 'same', kernel_initializer = init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(128, (3, 3), (1, 1), padding = 'same', kernel_initializer = init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(256, (3, 3), (2, 2), padding = 'same', kernel_initializer = init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Conv2D(512, (3, 3), (1, 1), padding = 'same', kernel_initializer = init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    y1 = layers.Dense(1, 'sigmoid')(x)
    y2 = layers.Dense(num_classes, 'softmax')(x)
    model = models.Model(inputs = [input, true_val], outputs = [y1, y2])
    return model

In [ ]:
gen = get_generator(11)
gen.summary()
plot_model(gen, show_shapes = True, dpi = 64, rankdir='LR')

In [ ]:
dis = get_discriminator(11)
dis.summary()
plot_model(dis, show_shapes = True, dpi = 50)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import tensorflow
loss_object_binary=tensorflow.keras.losses.BinaryCrossentropy(from_logits=True)
loss_object_categorical = tensorflow.keras.losses.SparseCategoricalCrossentropy()

In [ ]:
def discriminator_loss(disc_real_class, input_label, disc_generated_class, disc_generated_output, disc_real_output):
  
  bloss=loss_object_binary(tf.ones_like(disc_real_output), disc_real_output)+loss_object_binary(tf.zeros_like(disc_generated_output),disc_generated_output)
  closs=loss_object_categorical(input_label, disc_real_class)+loss_object_categorical(input_label, disc_generated_class)
  return closs+bloss

In [ ]:
def generator_loss(disc_generated_output, disc_generated_class, input_label):
  gan_loss = loss_object_binary(tf.ones_like(disc_generated_output), disc_generated_output)
  l1_loss = loss_object_categorical(input_label, disc_generated_class)
  total_gen_loss = gan_loss + l1_loss
  return total_gen_loss

In [ ]:
def train_step(generator, discriminator, input_image, input_label):
  global current
  BATCH_SIZE=len(input_image)
  noise_dim=110
  noise = tf.random.normal([BATCH_SIZE, noise_dim])
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator([noise, input_label], training=True)

    disc_real_output, disc_real_class = discriminator([input_image, input_label], training=True)
    disc_generated_output, disc_generated_class = discriminator( [gen_output,input_label], training=True)

    gen_total_loss = generator_loss(disc_generated_output, disc_generated_class, input_label)
    disc_total_loss = discriminator_loss(disc_real_class, input_label, disc_generated_class, disc_generated_output, disc_real_output)
    
  
  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_total_loss,
                                               discriminator.trainable_variables)
  print("GenL", gen_total_loss)
  print("DisL", disc_total_loss)
  generator_optimizer = optimizers.Adam(0.0002, 0.5, 0.999)
  discriminator_optimizer = optimizers.Adam(0.0002, 0.5, 0.999)
  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def print_image(step, gen, model_num):
  BATCH_SIZE=25
  noise_dim=110
  noise = tf.random.normal([BATCH_SIZE, noise_dim])
  label = tf.random.uniform((BATCH_SIZE, ), 0, 9, dtype = tf.dtypes.int64)
  checker = gen([noise, tf.dtypes.cast(label, tf.float32)], training=False)
  image = np.array(checker)
  image = (image + 1) / 2.0
  for i in range(BATCH_SIZE):
  # define subplot
    plt.subplot(5, 5, 1 + i)
    # turn off axis
    plt.axis('off')
    # plot raw pixel data
    plt.imshow(image[i, :, :, 0], cmap='gray_r')
  # save plot to file
  filename1 = 'output_images/' + str(model_num) + '/generated_plot_%04d.png' % (step+1)
  plt.savefig(filename1)
  plt.close()
def train_function(gen, disc, dataset, num_epochs, model_num):
  count = len(dataset)
  for j in range(num_epochs):
    if (j % 100 == 0):
      print("Current model number: ", model_num)
    i = 0
    print("Epoch number: ", j)
    for i in  range(len(dataset)):
      print("Batch: ", i)
      batch = dataset[i]
      image, label = batch
      train_step(gen, disc, image, label)

In [ ]:
gen_list = []
disc_list = []
for i in range(17):
  gen_list.append(get_generator(class_num_for_folder[i]))
  disc_list.append(get_discriminator(class_num_for_folder[i]))

In [ ]:
def print_image_with_class(cname, gen):
  BATCH_SIZE=25
  noise_dim=110
  noise = tf.random.normal([BATCH_SIZE, noise_dim])
  label = tf.zeros([BATCH_SIZE]) + cname;
  checker = gen([noise, tf.dtypes.cast(label, tf.float32)], training=False)
  image = np.array(checker)
  image = (image + 1) / 2.0
  for i in range(BATCH_SIZE):
  # define subplot
    plt.subplot(5, 5, 1 + i)
    # turn off axis
    plt.axis('off')
    # plot raw pixel data
    plt.imshow(image[i, :, :, 0], cmap='gray_r')
  # save plot to file
  filename1 = 'output_images_with_class/generated_plot_%04d.png' % (cname)
  plt.savefig(filename1)
  plt.close()

In [ ]:
def print_image_of_batch(dataset, idx):
  image, labels=dataset[idx]
  image = (image + 1) / 2.0
  print(labels)
  for i in range(20):
  # define subplot
    plt.subplot(5, 4, 1 + i)
    # turn off axis
    plt.axis('off')
    # plot raw pixel data
    plt.imshow(image[i, :, :, 0], cmap='gray_r')

In [ ]:
from google.colab import files
def save_checkpoint(gen, disc, model_num, epoch_num):
  gen_path = "ModelList/GenList/"
  disc_path = "ModelList/DiscList/"
  gen_fin = gen_path + "gen" + str(model_num) + str(epoch_num)
  disc_fin = disc_path + "disc" + str(model_num) + str(epoch_num)
  gen.save(gen_fin)
  disc.save(disc_fin)
  !zip -r loss_rev_acgan.zip ModelList
  !cp loss_rev_acgan.zip gdrive/MyDrive/


for i in range(0,17):
  train_function(gen_list[i], disc_list[i], train_gen_list[i], 100, i + 1)
  save_checkpoint(gen_list[i], disc_list[i], i + 1, 100)
  train_function(gen_list[i], disc_list[i], train_gen_list[i], 100, i + 1)
  save_checkpoint(gen_list[i], disc_list[i], i + 1, 200)
  train_function(gen_list[i], disc_list[i], train_gen_list[i], 100, i + 1)
  save_checkpoint(gen_list[i], disc_list[i], i + 1, 300)
  train_function(gen_list[i], disc_list[i], train_gen_list[i], 100, i + 1)
  save_checkpoint(gen_list[i], disc_list[i], i + 1, 400)


Current model number:  1
Epoch number:  0
Batch:  0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


GenL tf.Tensor(3.853245, shape=(), dtype=float32)
DisL tf.Tensor(8.139037, shape=(), dtype=float32)
Batch:  1
GenL tf.Tensor(4.074783, shape=(), dtype=float32)
DisL tf.Tensor(8.054093, shape=(), dtype=float32)
Batch:  2
GenL tf.Tensor(4.503648, shape=(), dtype=float32)
DisL tf.Tensor(7.6141815, shape=(), dtype=float32)
Batch:  3
GenL tf.Tensor(3.8657193, shape=(), dtype=float32)
DisL tf.Tensor(8.073227, shape=(), dtype=float32)
Batch:  4
GenL tf.Tensor(4.7827873, shape=(), dtype=float32)
DisL tf.Tensor(8.512268, shape=(), dtype=float32)
Batch:  5
GenL tf.Tensor(3.706102, shape=(), dtype=float32)
DisL tf.Tensor(7.9209285, shape=(), dtype=float32)
Batch:  6
GenL tf.Tensor(5.1189117, shape=(), dtype=float32)
DisL tf.Tensor(9.054937, shape=(), dtype=float32)
Batch:  7
GenL tf.Tensor(3.675059, shape=(), dtype=float32)
DisL tf.Tensor(7.7170424, shape=(), dtype=float32)
Batch:  8
GenL tf.Tensor(4.703533, shape=(), dtype=float32)
DisL tf.Tensor(8.093805, shape=(), dtype=float32)
Batch:  9
GenL

OSError: ignored

In [ ]:
def get_checkpoint(model_num, epoch_num):
  gen_path = "ModelList/GenList/"
  disc_path = "ModelList/DiscList/"
  gen_fin = gen_path + "gen" + str(model_num) + str(epoch_num)
  disc_fin = disc_path + "disc" + str(model_num) + str(epoch_num)
  gen = keras.models.load_model(gen_fin)
  disc = keras.models.load_model(disc_fin)
  return gen, disc
def load_model_now():
  !cp gdrive/MyDrive/random_ccgan.zip ./
  !unzip random_ccgan.zip -d ./
  for i in range(0,17):
    try:
      gen_list[i], disc_list[i] = get_checkpoint(i + 1, 100)
      gen_list[i], disc_list[i] = get_checkpoint(i + 1, 200)
      gen_list[i], disc_list[i] = get_checkpoint(i + 1, 400)
    except:
      pass
load_model_now()

Archive:  random_ccgan.zip
replace ./ModelList/GenList/gen2100/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./ModelList/GenList/gen2100/saved_model.pb  
  inflating: ./ModelList/GenList/gen2100/variables/variables.data-00000-of-00001  
  inflating: ./ModelList/GenList/gen2100/variables/variables.index  
  inflating: ./ModelList/GenList/gen2100/keras_metadata.pb  
  inflating: ./ModelList/DiscList/disc2100/saved_model.pb  
  inflating: ./ModelList/DiscList/disc2100/variables/variables.data-00000-of-00001  
  inflating: ./ModelList/DiscList/disc2100/variables/variables.index  
  inflating: ./ModelList/DiscList/disc2100/keras_metadata.pb  
   creating: ./ModelList/GenList/gen2200/
  inflating: ./ModelList/GenList/gen2200/saved_model.pb  
   creating: ./ModelList/GenList/gen2200/variables/
  inflating: ./ModelList/GenList/gen2200/variables/variables.data-00000-of-00001  
  inflating: ./ModelList/GenList/gen2200/variables/variables.index  
   creating: ./ModelList/Ge